In [48]:
import pandas as pd
from scipy import *
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import *

In [15]:
import implicit

In [33]:
PLIDs = load("all_45k_playlist_ordered_by_ID.npy")
test = sparse.load_npz("all_playlists_with_tracks.npz")
TIDs = load("all_100k_tracks_ordered_by_ID.npy")

PLIDs = ravel(PLIDs)[0].getcol(0).data

In [24]:
def implicit_weighted_ALS(training_set, lambda_val = 0.1, alpha = 40, iterations = 15, rank_size = 30, seed = 0):
    '''
    Implicit weighted ALS taken from Hu, Koren, and Volinsky 2008. Designed for alternating least squares and implicit
    feedback based collaborative filtering. 
    
    parameters:
    
    training_set - Our matrix of ratings with shape m x n, where m is the number of users and n is the number of items.
    Should be a sparse csr matrix to save space. 
    
    lambda_val - Used for regularization during alternating least squares. Increasing this value may increase bias
    but decrease variance. Default is 0.1. 
    
    alpha - The parameter associated with the confidence matrix discussed in the paper, where Cui = 1 + alpha*Rui. 
    The paper found a default of 40 most effective. Decreasing this will decrease the variability in confidence between
    various ratings.
    
    iterations - The number of times to alternate between both user feature vector and item feature vector in
    alternating least squares. More iterations will allow better convergence at the cost of increased computation. 
    The authors found 10 iterations was sufficient, but more may be required to converge. 
    
    rank_size - The number of latent features in the user/item feature vectors. The paper recommends varying this 
    between 20-200. Increasing the number of features may overfit but could reduce bias. 
    
    seed - Set the seed for reproducible results
    
    returns:
    
    The feature vectors for users and items. The dot product of these feature vectors should give you the expected 
    "rating" at each point in your original matrix. 
    '''
    
    # first set up our confidence matrix
    
    conf = (alpha*training_set) # To allow the matrix to stay sparse, I will add one later when each row is taken 
                                # and converted to dense. 
    num_user = conf.shape[0]
    num_item = conf.shape[1] # Get the size of our original ratings matrix, m x n
    
    # initialize our X/Y feature vectors randomly with a set seed
    rstate = np.random.RandomState(seed)
    
    X = sparse.csr_matrix(rstate.normal(size = (num_user, rank_size))) # Random numbers in a m x rank shape
    Y = sparse.csr_matrix(rstate.normal(size = (num_item, rank_size))) # Normally this would be rank x n but we can 
                                                                 # transpose at the end. Makes calculation more simple.
    X_eye = sparse.eye(num_user)
    Y_eye = sparse.eye(num_item)
    lambda_eye = lambda_val * sparse.eye(rank_size) # Our regularization term lambda*I. 
    
    # We can compute this before iteration starts. 
    
    # Begin iterations
   
    for iter_step in range(iterations): # Iterate back and forth between solving X given fixed Y and vice versa
        # Compute yTy and xTx at beginning of each iteration to save computing time
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)
        # Being iteration to solve for X based on fixed Y
        for u in range(num_user):
            conf_samp = conf[u,:].toarray() # Grab user row from confidence matrix and convert to dense
            pref = conf_samp.copy() 
            pref[pref != 0] = 1 # Create binarized preference vector 
            CuI = sparse.diags(conf_samp, [0]) # Get Cu - I term, don't need to subtract 1 since we never added it 
            yTCuIY = Y.T.dot(CuI).dot(Y) # This is the yT(Cu-I)Y term 
            yTCupu = Y.T.dot(CuI + Y_eye).dot(pref.T) # This is the yTCuPu term, where we add the eye back in
                                                      # Cu - I + I = Cu
            X[u] = spsolve(yTy + yTCuIY + lambda_eye, yTCupu) 
            # Solve for Xu = ((yTy + yT(Cu-I)Y + lambda*I)^-1)yTCuPu, equation 4 from the paper  
        # Begin iteration to solve for Y based on fixed X 
        for i in range(num_item):
            conf_samp = conf[:,i].T.toarray() # transpose to get it in row format and convert to dense
            pref = conf_samp.copy()
            pref[pref != 0] = 1 # Create binarized preference vector
            CiI = sparse.diags(conf_samp, [0]) # Get Ci - I term, don't need to subtract 1 since we never added it
            xTCiIX = X.T.dot(CiI).dot(X) # This is the xT(Cu-I)X term
            xTCiPi = X.T.dot(CiI + X_eye).dot(pref.T) # This is the xTCiPi term
            Y[i] = spsolve(xTx + xTCiIX + lambda_eye, xTCiPi)
            # Solve for Yi = ((xTx + xT(Cu-I)X) + lambda*I)^-1)xTCiPi, equation 5 from the paper
    # End iterations
    return X, Y.T # Transpose at the end to make up for not being transposed at the beginning. 
                         # Y needs to be rank x n. Keep these as separate matrices for scale reasons. 

In [76]:
urm = sparse.load_npz("all_playlist_with_tracks_URM.npz")

urm_train = urm[invert(isin(PLIDs, target_playlists))]
urm_test = urm[isin(PLIDs, target_playlists)]
urm_test = urm_test[:, isin(TIDs, target_tracks_ordered)]

In [77]:
urm_train

<35649x100000 sparse matrix of type '<class 'numpy.int32'>'
	with 677861 stored elements in Compressed Sparse Column format>

In [79]:
alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((urm_train*alpha).astype('double'), 
                                                          factors=30, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

alpha = 15
user_vecs2, item_vecs2 = implicit.alternating_least_squares((urm_test*alpha).astype('double'), 
                                                          factors=30, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

In [37]:
pl_test = user_vecs[isin(PLIDs, target_playlists)]


In [80]:
monoArtist=load("target_playlist_mono_artist.npy")
monoArtistPlID=monoArtist[:,0]
monoArtistArtistID=monoArtist[:,1]
artistIndexes=load("uniqueArtists_NeededToIndexThe_ArtistReducedMatrices.npy")
artistTracks=sparse.load_npz("artists_with_tracksID_ordered_by_occurrencies.npz")
target_playlists=genfromtxt("target_playlists.csv",skip_header=1)
playlists_with_tracks=load("playlists_with_tracks.npy")
target_tracks_ordered = load("targetTracksOrdered.npy")
def getsimil(pls, similrow, n):
    maxi = flip(argsort(similrow), axis=0)
    r = []
    for m in maxi:
        if(not isin(target_tracks_ordered[m], pls[1:])):
           r.append(target_tracks_ordered[m])
           if(len(r)==n):
               return r

In [67]:
item_test = item_vecs[isin(TIDs, target_tracks_ordered)]

In [81]:
sim = user_vecs2.dot(item_test.T)

In [82]:
def compute(fname, finalPlSim):
    with open(fname,"a") as myfile:
        myfile.write("playlist_id,track_ids\n")
        for pl, similsum in zip(playlists_with_tracks, finalPlSim):
            plID=pl[0]
            s = str(int(plID))
            s += ","
            if(isin(plID,monoArtistPlID)):#guarda se c'è un artista solo
                artist=monoArtistArtistID[where(monoArtistPlID==plID)[0][0]]#l'unico artista della playlist
                artistIndex=where(artistIndexes==artist)[0][0]#cerca la sua posizione nella matrice delle tracce
                thisArtistTracks=artistTracks.getrow(artistIndex).data#prende le sue tracce
                rr=array([t for t in thisArtistTracks if t not in pl[1:] and isin(t, target_tracks_ordered)])#prende tutte quelle non già presenti

                if(len(rr)>=5):
                    r = array(rr).take(range(5))
                else:
                    r4 = getsimil(pl, ravel(similsum), 5)
                    r4 = array([el for el in r4 if el not in rr])
                    r = concatenate((rr, r4.take(range(5-len(rr)))))
            else:
                r = getsimil(pl, ravel(similsum), 5)#non c'è un artista solo quindi guarda solo i tag
            for el in r:
                s+=str(el)
                s+=" "
            myfile.write(s + "\n")

In [83]:
PlwithArtist = sparse.load_npz("targetPlaylistArtistReduced.npz")
PlwithArtist = normalize(PlwithArtist, norm="max")
tTracksWithArtist = sparse.load_npz("targetTracksArtistReduced.npz")
PlTrackSimByArtist = PlwithArtist * tTracksWithArtist.T

In [84]:
fin = sim + 0.75*PlTrackSimByArtist

In [85]:
compute("ALSonlytrain.csv", fin)